In [1]:
! pip install transformers datasets evaluate scikit-learn
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.0 MB/s eta 0:00:00


In [2]:
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda:0


In [5]:
from datasets import load_dataset
ds= load_dataset("SetFit/sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
})

In [7]:
ds['train'][0]

{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films',
 'label': 1,
 'label_text': 'positive'}

In [8]:
from transformers import AutoTokenizer #finetuning turotial

bert_cased_tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

text = 'Hello everyone! ! antidisestablishmentarianism'

tokenizer=bert_cased_tokenizer

vocab = {v: k for k, v in tokenizer.vocab.items()}
tokenized_text = tokenizer(text)
print([vocab[id] for id in tokenized_text['input_ids']])

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

['[CLS]', 'Hello', 'everyone', '!', '!', 'anti', '##dis', '##esta', '##b', '##lish', '##ment', '##arian', '##ism', '[SEP]']


In [9]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [10]:
tokenized_sst=ds.map(preprocess_function,batched=True)

Map:   0%|          | 0/6920 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import evaluate

# Proportion of correct predictions among the total number of cases processed
accuracy = evaluate.load('accuracy')

In [13]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
labels = ['NEGATIVE', 'POSITIVE']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

print('id2label:', id2label)
print('label2id:', label2id)

id2label: {0: 'NEGATIVE', 1: 'POSITIVE'}
label2id: {'NEGATIVE': 0, 'POSITIVE': 1}


In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# This automodel class gives us the model with pretrained weights + a sequence classification head
# We specify how many labels we need so that the model has the correct number of outputs
# We specify id2label/label2id so that the model understands the label associated with each output
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=len(labels), id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir='my_awesome_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
) #finetuningtutorial

In [17]:
import time
import math
from tqdm import tqdm
import wandb #finetuning tutorial,chatgpt

class CustomTrainer(Trainer):
  def _inner_training_loop(
      self, batch_size=None, args=None, resume_from_checkpoint=None, trial=None, ignore_keys_for_eval=None
  ):
    number_of_epochs = args.num_train_epochs
    start=time.time()
    train_loss=[]
    train_acc=[]
    eval_acc=[]

    criterion=torch.nn.CrossEntropyLoss().to(device)
    self.optimizer=torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    self.scheduler=torch.optim.lr_scheduler.StepLR(self.optimizer,1,gamma=0.9)

    learning_rate=args.learning_rate
    num_train_epochs=args.num_train_epochs

    run=wandb.init(
        project='my_awesome_model',

        config={
            "learning_rate": learning_rate,
            "epochs":num_train_epochs
        }
    )
    train_dataloader= self.get_train_dataloader()
    eval_dataloader= self.get_eval_dataloader()

    max_steps= math.ceil(args.num_train_epochs * len(train_dataloader))

    for epoch in range(number_of_epochs):
      train_loss_per_epoch =0
      train_acc_per_epoch=0
      with tqdm(train_dataloader, unit="batch") as training_epoch:
        training_epoch.set_description(f"Training Epoch {epoch}")
        for step, inputs in enumerate(training_epoch):
          inputs= inputs.to(device)
          labels= inputs['labels']
          #forward pass have to do

          #output=.. todo
          inputs={k: v.to(device) for k, v in inputs.items()}
          output=self.model(**inputs)

          #getloss
          #loss=criterion((output[???],labels)) to do
          loss=criterion(output.logits, inputs['labels'])

          train_loss_per_epoch += loss.item()

          #have to calculate gradients
          self.optimizer.zero_grad()
          loss.backward()
          #update weights
          self.optimizer.step()
          train_acc_per_epoch += (output['logits'].argmax(1)== labels).sum().item()

          #adjust the learning rate

        self.scheduler.step()
        train_loss_per_epoch /= len(train_dataloader)
        train_acc_per_epoch /= (len(train_dataloader)*batch_size)
        wandb.log({"train accuracy":train_acc_per_epoch,"\t train loss":train_loss_per_epoch})

        eval_loss_per_epoch=0
        eval_acc_per_epoch =0
        with tqdm(eval_dataloader, unit="batch") as eval_epoch:
          eval_epoch.set_description(f"Evaluation Epoch {epoch}")
          #to do

          for valuez in eval_epoch:
            valuez=valuez.to(device)
            labels=valuez['labels']
            valuez={k: v.to(device) for k,v in valuez.items()}
            eval_output=self.model(**valuez)

            eval_loss=criterion(eval_output.logits,valuez['labels'])
            eval_loss_per_epoch+=eval_loss.item()
            eval_pred=eval_output.logits.argmax(dim=-1)
            eval_acc_per_epoch+= (eval_pred==valuez['labels']).sum().item()
        eval_loss_per_epoch /= (len(eval_dataloader))
        eval_acc_per_epoch /= (len(eval_dataloader)*batch_size)
        wandb.log({"eval accuracy":eval_acc_per_epoch,"\t eval loss":eval_loss_per_epoch})

        print(f'\tTrain Loss: {train_loss_per_epoch: .3f} | Train Acc: {train_acc_per_epoch*100: .2f}%')
        print(f'\tEval Loss: {eval_loss_per_epoch: .3f} | Eval Acc: {eval_acc_per_epoch*100: .2f}%')
      print(f'Time: {(time.time()-start)/60: .3f} minutes')




In [18]:
trainer=CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst['train'],
    eval_dataset=tokenized_sst['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: vamsikoneru. Use `wandb login --relogin` to force relogin


Evaluation Epoch 0: 100%|██████████| 28/28 [00:00<00:00, 30.95batch/s]


	Train Loss:  0.345 | Train Acc:  84.86%
	Eval Loss:  0.249 | Eval Acc:  88.62%
Time:  0.420 minutes


Evaluation Epoch 1: 100%|██████████| 28/28 [00:00<00:00, 30.64batch/s]


	Train Loss:  0.111 | Train Acc:  96.36%
	Eval Loss:  0.262 | Eval Acc:  87.61%
Time:  0.792 minutes


...

	Train Loss:  0.030 | Train Acc:  98.69%
	Eval Loss:  0.400 | Eval Acc:  87.61%
Time:  1.164 minutes


Evaluation Epoch 3: 100%|██████████| 28/28 [00:00<00:00, 32.63batch/s]


	Train Loss:  0.012 | Train Acc:  99.37%
	Eval Loss:  0.469 | Eval Acc:  87.05%
Time:  1.553 minutes


Evaluation Epoch 4: 100%|██████████| 28/28 [00:00<00:00, 32.32batch/s]

	Train Loss:  0.005 | Train Acc:  99.54%
	Eval Loss:  0.489 | Eval Acc:  87.95%
Time:  1.928 minutes


In [19]:
tokenized_sst

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [20]:
from transformers import pipeline
import time
#chatgpt
inf_start=time.time()
classifier=pipeline('sentiment-analysis',model=model,tokenizer=tokenizer,device=device)
test_set= tokenized_sst['test']
#for testing in test_set:
 # labels=testing['labels']
total_samples = 0
correct_predictions = 0


for testing in test_set:
    #print(f"{testing['text']}\t {testing['label']}")
    text = testing['text']

    result = classifier(text)

    predicted_label = result[0]['label']

    def texttoval(a):
      if a=='POSITIVE':
        return 1
      else:
        return 0
    count=0
    if texttoval(predicted_label) == testing['label']:
        correct_predictions += 1
    else:
      print(f"{testing['text']}\n The predicted label: {predicted_label}, \tthe ground truth: {testing['label']}, \tconfidence score: {result[0]['score']}")
      count=count+1

    total_samples += 1

accuracy = correct_predictions / total_samples

print("Accuracy:", accuracy)
print(f'Time: {(time.time()-inf_start)/60: .3f} minutes')


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


acting , particularly by tambor , almost makes `` never again '' worthwhile , but -lrb- writer\/director -rrb- schaeffer should follow his titular advice
 The predicted label: POSITIVE, 	the ground truth: 0, 	confidence score: 0.7271924018859863
oft-described as the antidote to american pie-type sex comedies , it actually has a bundle in common with them , as the film diffuses every opportunity for a breakthrough
 The predicted label: POSITIVE, 	the ground truth: 0, 	confidence score: 0.993291974067688
an experience so engrossing it is like being buried in a new environment .
 The predicted label: NEGATIVE, 	the ground truth: 1, 	confidence score: 0.5053796172142029
a cockamamie tone poem pitched precipitously between swoony lyricism and violent catastrophe ... the most aggressively nerve-wracking and screamingly neurotic romantic comedy in cinema history .
 The predicted label: NEGATIVE, 	the ground truth: 1, 	confidence score: 0.9926181435585022
as are its star , its attitude and its